In [ ]:
import os
from PIL import Image
import numpy as np

def apply_gamma(image, gamma):
    """
    Apply gamma correction to an image using a lookup table.
    
    Args:
        image (PIL.Image): Input image
        gamma (float): Gamma value to apply
    
    Returns:
        PIL.Image: Gamma-adjusted image
    """
    lut = [int(((i / 255.0) ** gamma) * 255.0 + 0.5) for i in range(256)]
    channels = image.split()
    adjusted_channels = [channel.point(lut) for channel in channels]
    return Image.merge('RGB', adjusted_channels)

def compute_gamma(original_avg, target_avg, tolerance=5):
    """
    Compute the gamma value to adjust the image’s average intensity toward the target.
    
    Args:
        original_avg (float): Original average pixel intensity
        target_avg (float): Target average pixel intensity
        tolerance (float): Tolerance within which no adjustment is needed (default: 5)
    
    Returns:
        float: Computed gamma value
    """
    if abs(original_avg - target_avg) < tolerance:
        return 1.0  # No adjustment needed
    return np.log(target_avg / 255.0) / np.log(original_avg / 255.0)

def main(input_folder, output_folder, target_avg_start=50, target_avg_end=200, target_avg_step=10, tolerance=5):
    """
    Perform a grid search over target_avg values to adjust gamma of all images in a folder.
    
    Args:
        input_folder (str): Path to folder containing images
        output_folder (str): Path to save adjusted images
        target_avg_start (float): Starting target average intensity (default: 50)
        target_avg_end (float): Ending target average intensity (default: 150)
        target_avg_step (float): Step size for target average (default: 10)
        tolerance (float): Tolerance for no adjustment (default: 5)
    """
    # Validate input folder
    if not os.path.exists(input_folder):
        raise FileNotFoundError(f"Input folder not found: {input_folder}")
    
    # Create output folder if it doesn’t exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Define target_avg values for grid search
    target_avg_values = np.arange(target_avg_start, target_avg_end + target_avg_step, target_avg_step).tolist()
    
    # Output text file for results
    output_text_file = os.path.join(output_folder, "grid_search_results.txt")
    results = []
    
    # Supported image extensions
    image_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif')
    
    # Process each image in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(image_extensions):
            image_path = os.path.join(input_folder, filename)
            try:
                # Open and convert image to RGB
                image = Image.open(image_path).convert('RGB')
                original_array = np.array(image)
                
                # Calculate original average intensity
                original_avg = np.mean(original_array)
                
                # Process each target_avg value
                for target_avg in target_avg_values:
                    # Compute dynamic gamma for this target_avg
                    gamma = compute_gamma(original_avg, target_avg, tolerance)
                    
                    # Apply gamma correction
                    adjusted_image = apply_gamma(image, gamma)
                    
                    # Calculate adjusted average
                    adjusted_avg = np.mean(np.array(adjusted_image))
                    
                    # Generate output filename
                    base_name = os.path.splitext(filename)[0]
                    output_filename = f"{base_name}_target_avg_{int(target_avg)}.jpg"
                    output_path = os.path.join(output_folder, output_filename)
                    
                    # Save the adjusted image
                    adjusted_image.save(output_path)
                    
                    # Log result
                    result = (f"{filename}: Target avg: {target_avg}, Original avg: {original_avg:.2f}, "
                              f"Gamma: {gamma:.2f}, Adjusted avg: {adjusted_avg:.2f}")
                    results.append(result)
                    print(result)
                    
            except Exception as e:
                error_msg = f"Error processing {filename}: {str(e)}"
                results.append(error_msg)
                print(error_msg)
    
    # Save results to text file
    with open(output_text_file, "w") as f:
        for result in results:
            f.write(result + "\n")
    
    print(f"Grid search complete. Results saved to {output_text_file}")

if __name__ == "__main__":
    input_folder = "/home/chan87/Desktop/tmp3/tiles/M"  # Input folder path
    output_folder = "/home/chan87/Desktop/tmp3/tiles/M2"  # Output folder path
    main(input_folder, output_folder)

In [ ]:
import os
from PIL import Image
import numpy as np

def apply_gamma(image, gamma):
    """
    Apply gamma correction to an image using a lookup table.
    
    Args:
        image (PIL.Image): Input image
        gamma (float): Gamma value to apply
    
    Returns:
        PIL.Image: Gamma-adjusted image
    """
    lut = [int(((i / 255.0) ** gamma) * 255.0 + 0.5) for i in range(256)]
    channels = image.split()
    adjusted_channels = [channel.point(lut) for channel in channels]
    return Image.merge('RGB', adjusted_channels)

def compute_gamma(original_avg, target_avg=128, tolerance=5):
    """
    Compute the gamma value to adjust the image’s average intensity toward the target.
    
    Args:
        original_avg (float): Original average pixel intensity
        target_avg (float): Target average pixel intensity (default: 128)
        tolerance (float): Tolerance within which no adjustment is needed (default: 5)
    
    Returns:
        float: Computed gamma value
    """
    if abs(original_avg - target_avg) < tolerance:
        return 1.0  # No adjustment needed
    if original_avg < 1 or original_avg > 254:
        return 1.0  # Avoid extreme cases
    return np.log(target_avg / 255.0) / np.log(original_avg / 255.0)

def main(folder_path, target_avg=130, tolerance=5):
    """
    Adjust gamma of all images in a folder dynamically based on their original average intensity.
    
    Args:
        folder_path (str): Path to folder containing images
        target_avg (float): Target average pixel intensity (default: 128)
        tolerance (float): Tolerance for no adjustment (default: 5)
    """
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"Folder not found: {folder_path}")
    
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image_path = os.path.join(folder_path, filename)
            try:
                # Open and convert image to RGB
                image = Image.open(image_path).convert('RGB')
                original_array = np.array(image)
                
                # Calculate original average intensity
                original_avg = np.mean(original_array)
                
                # Compute dynamic gamma for this image
                gamma = compute_gamma(original_avg, target_avg, tolerance)
                
                # Apply gamma correction
                adjusted_image = apply_gamma(image, gamma)
                
                # Save the adjusted image (overwrites original)
                adjusted_image.save(image_path)
                
                # Calculate and report the adjusted average
                adjusted_avg = np.mean(np.array(adjusted_image))
                print(f"{filename}: Original avg: {original_avg:.2f}, Gamma: {gamma:.2f}, Adjusted avg: {adjusted_avg:.2f}")
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")

if __name__ == "__main__":
    folder_path = "/home/chan87/Desktop/tmp3/tiles/M2"  # Adjust this path as needed
    main(folder_path)